In [35]:
# [IMPORTS]
! pip install pandas
! pip install openpyxl
import pandas as pd


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [36]:

# [FONCTIONS]

# Fonction de renommage de nom de colonne 
def rename_columns(data, list_dico):
  for dico in list_dico:
    data.rename(columns= dico, inplace=True)

# Fonction de suppression de colonnes inutile
def drop_columns(data, list_dico_col_renam, joker):

  # List des colonnes à supprimer dans le COG commune
  drop_list_com= [col for col in data.columns if [col] not in [list(dico.values()) for dico in list_dico_col_renam] and col!= joker]
  
  # Suppression des colonnes 
  data= data.drop(columns= drop_list_com)
  return data


# Fonction de lecture et de traitement des fichiers COG
def read_and_clean(file_name, list_dico_rename, except_col):

  # Lecture de la BDD
  data= pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/COG_FRANCE/{file_name}.csv")
  data.columns= [col.lower() for col in data.columns]

  # Renommage des colonnes
  rename_columns(data,list_dico_rename)
  data=  drop_columns(data, list_dico_rename, except_col)
  return data

- **Traitement des données COG relatives aux regions, département, arrondissement et communes**

In [37]:
#-   Traitement des données relatives aux regions

COG_region= read_and_clean(file_name= "v_region_2025", list_dico_rename= [{"reg": "code_reg"}], except_col= "lib_reg")

#-   Traitement des données relatives aux départements

COG_departement= read_and_clean(file_name= "v_departement_2025", list_dico_rename= [{"dep": "code_dept"}], except_col= "lib_dept")


#-   Traitement des données relatives aux arrondissement
COG_arrondissement= read_and_clean(file_name= "v_arrondissement_2025", list_dico_rename= [{"arr": "code_arr"}], except_col= "lib_arr")
  
#-   Traitement des données relatives aux communes

COG_commune= read_and_clean(file_name= "v_commune_2025", list_dico_rename= [
  {"com": "code_com"},
  {"reg": "code_reg"},
  {"dep": "code_dept"},
  {"arr": "code_arr"}], except_col= "lib_com")

#+ Jointure des tables

# Jointure commune - arrondissement
COG_com_arr=  COG_commune.merge(COG_arrondissement, on= "code_arr", how= "left") 

#Jointure commune - arrondissement - departement
COG_com_arr_dept= COG_com_arr.merge(COG_departement, on= "code_dept", how= "left") 

# Jointure commune - arrondissement - departement - region
COG_com_arr_dept_reg= COG_com_arr_dept.merge(COG_region, on= "code_reg", how= "left") 

#+ Filtre Bretagne
# Filtre sur la région de Bretagne
COG_Bretagne= COG_com_arr_dept_reg[COG_com_arr_dept_reg.code_reg== 53]

COG_Bretagne.head() #visualisation

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg
7784,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne
7785,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne
7786,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne
7787,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne
7788,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne


- **Lecture et jointure des données liée à la densité de la population au sein de la commune: griden7**

In [38]:
densite_data = pd.read_excel(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/DENSITE/densite.xlsx", skiprows= 4)

densite_data= densite_data.rename(columns= {"codgeo": "code_com"})
densite_data= densite_data.drop(columns= "libgeo")
Dataframe_stage_1= COG_Bretagne.merge(densite_data, on= "code_com", how= "left") 

Dataframe_stage_1.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5


+ **Lecture et jointure des données liée du nom d'habitant dans la commune: p_pop**

In [39]:
population_data = pd.read_excel(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/POPULATION/population.xlsx", skiprows= 4)

recent_year_for_p_pop= max(list(population_data.an))

population_data= population_data[population_data.an== recent_year_for_p_pop ]
population_data= population_data.rename(columns= {"codgeo": "code_com"})

population_data= population_data[["code_com", "p_pop"]]

Dataframe_stage_2= Dataframe_stage_1.merge(population_data, on= "code_com", how= "left") 

Dataframe_stage_2.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0


+ **Lecture et jointure des données liée à la superficie: superf_choro**

In [40]:
superficie_data = pd.read_excel(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/OCCUPATION_DES_SOLS/superficie.xlsx", skiprows= 4)

recent_year_for_superficie= max(list(superficie_data.an))

superficie_data= superficie_data[superficie_data.an== recent_year_for_superficie]
superficie_data= superficie_data.rename(columns= {"codgeo": "code_com"})

superficie_data= superficie_data[["code_com", "superf_choro"]]

Dataframe_stage_3= Dataframe_stage_2.merge(superficie_data, on= "code_com", how= "left") 

Dataframe_stage_3.head()


,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,superf_choro
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,24.1
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,12.2
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,6.4
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,36.4
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,14.1


+ **Lecture et jointure des données liée à la superficie :gare_tgv**

In [41]:
gare_tgv_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/TRANSPORT_FERROVIAIRE/arret_ferroviaire.csv")

gare_tgv_data= gare_tgv_data.rename(columns= {"code_insee": "code_com"})

gare_tgv_data= gare_tgv_data.rename(columns= {"tgv": "gare_tgv"})

gare_tgv_data= gare_tgv_data[["code_com", "gare_tgv"]]

gare_tgv_data["gare_tgv"]= gare_tgv_data["gare_tgv"].map(lambda dec: (1 if dec == "O" else 0) if dec in ["O", "N"] else pd.NA)

gare_tgv_data_group = gare_tgv_data.groupby("code_com", as_index=False).sum()

gare_tgv_data_group['code_com'] = gare_tgv_data_group['code_com'].astype(str)

Dataframe_stage_4= Dataframe_stage_3.merge(gare_tgv_data_group, on= "code_com", how= "left") 

Dataframe_stage_4["gare_tgv"] = pd.to_numeric(Dataframe_stage_4["gare_tgv"], errors='coerce').fillna(0)

Dataframe_stage_4["gare_tgv"] = Dataframe_stage_4["gare_tgv"].astype(int)

Dataframe_stage_4.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,superf_choro,gare_tgv
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,24.1,0
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,12.2,0
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,6.4,0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,36.4,0
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,14.1,0


+ **Lecture et jointure des données liée aux actes france renov: nb_actes_france_renov**

In [42]:
actes_fr_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/ACTES_FRANCE_RENOV/actes_fr.csv")

actes_fr_data= actes_fr_data.rename(columns= {"codeInsee": "code_com"})

actes_fr_data= actes_fr_data.rename(columns= {"nb_actes" :"nb_actes_france_renov"}) 

actes_fr_data= actes_fr_data[["code_com", "nb_actes_france_renov"]]

actes_fr_data['code_com'] = actes_fr_data['code_com'].astype(str)

actes_fr_data['nb_actes_france_renov'] = actes_fr_data['nb_actes_france_renov'].astype(int)

group_com_acte_data = actes_fr_data.groupby("code_com", as_index=False).sum()

Dataframe_stage_5= Dataframe_stage_4.merge(group_com_acte_data, on= "code_com", how= "left") 

Dataframe_stage_5['nb_actes_france_renov']= Dataframe_stage_5['nb_actes_france_renov'].fillna(0)

Dataframe_stage_5.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,superf_choro,gare_tgv,nb_actes_france_renov
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,24.1,0,6.0
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,12.2,0,0.0
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,6.4,0,8.0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,36.4,0,4.0
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,14.1,0,1.0


+ **Lecture et jointure des données liée aux friches : friches**

In [43]:

friches_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/FRICHES/friches_france.csv")

friches_data = friches_data.rename(columns= {"comm_insee": "code_com"})

friches_data = friches_data[["code_com", "code_com"]]

friches_data.columns= ["code_com", "friches"]

friches_data['code_com'] = friches_data['code_com'].astype(str)
 
group_friches_data = friches_data.groupby("code_com", as_index=False).count()

Dataframe_stage_6= Dataframe_stage_5.merge(group_friches_data, on= "code_com", how= "left") 

Dataframe_stage_6["friches"] = Dataframe_stage_6["friches"].fillna(0)
Dataframe_stage_6.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,superf_choro,gare_tgv,nb_actes_france_renov,friches
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,24.1,0,6.0,2.0
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,12.2,0,0.0,0.0
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,6.4,0,8.0,0.0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,36.4,0,4.0,6.0
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,14.1,0,1.0,3.0


+ **Lecture et jointure des données liée aux ecoquartier: ecoquartiers**

In [44]:
eco_quartier_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/ECO_QUARTIER/quartier.csv")

eco_quartier_data= eco_quartier_data.rename(columns= {"Insee commune": "code_com"})

eco_quartier_data["ecoquartiers"]= 1

eco_quartier_data= eco_quartier_data[["code_com","ecoquartiers"]]

eco_quartier_data['code_com'] = eco_quartier_data['code_com'].astype(str)
 
group_eco_quartier_data = eco_quartier_data.groupby("code_com", as_index=False).count()

Dataframe_stage_7= Dataframe_stage_6.merge(group_eco_quartier_data, on= "code_com", how= "left") 

Dataframe_stage_7["ecoquartiers"] = Dataframe_stage_7["ecoquartiers"].fillna(0)

Dataframe_stage_7.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,superf_choro,gare_tgv,nb_actes_france_renov,friches,ecoquartiers
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,24.1,0,6.0,2.0,0.0
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,12.2,0,0.0,0.0,0.0
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,6.4,0,8.0,0.0,0.0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,36.4,0,4.0,6.0,0.0
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,14.1,0,1.0,3.0,0.0


+ **Lecture et jointure des données liée aux parts d'actifs et inactifs part_actifs & part_inactifs**

In [45]:
# Actifs
actifs_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/ACTIFS_INACTIFS/actifs.csv", delimiter= ";", skiprows= 2)

lib_for_rename_1= [col for col in actifs_data.columns if col.startswith("Nombre d'actif")][0]

actifs_data= actifs_data.rename(columns= {"Code": "code_com", lib_for_rename_1 : "part_actifs"})

actifs_data= actifs_data[["code_com", "part_actifs"]]

actifs_data['code_com'] = actifs_data['code_com'].astype(str)


# Inactifs

inactifs_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/ACTIFS_INACTIFS/inactifs.csv", delimiter= ";", skiprows= 2)

lib_for_rename_2= [col for col in inactifs_data.columns if col.startswith("Nombre d'inactif")][0]

inactifs_data= inactifs_data.rename(columns= {"Code": "code_com", lib_for_rename_2 : "part_inactifs"})

inactifs_data = inactifs_data[["code_com", "part_inactifs"]]

inactifs_data['code_com'] = inactifs_data['code_com'].astype(str)

Dataframe_stage_8= Dataframe_stage_7.merge(actifs_data, on= "code_com", how= "left") 
Dataframe_stage_9= Dataframe_stage_8.merge(inactifs_data, on= "code_com", how= "left") 

Dataframe_stage_9.replace("N/A - résultat non disponible", pd.NA, inplace=True)

Dataframe_stage_9[['part_actifs', 'part_inactifs']] = Dataframe_stage_9[['part_actifs', 'part_inactifs']].astype('int')

Dataframe_stage_9.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,superf_choro,gare_tgv,nb_actes_france_renov,friches,ecoquartiers,part_actifs,part_inactifs
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,24.1,0,6.0,2.0,0.0,281,76
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,12.2,0,0.0,0.0,0.0,584,128
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,6.4,0,8.0,0.0,0.0,453,133
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,36.4,0,4.0,6.0,0.0,1962,834
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,14.1,0,1.0,3.0,0.0,437,146


+ **Lecture et jointure des données liée aux revenu median med_disp**

In [46]:

revenu_med_data = pd.read_excel(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/REVENU_MED/revenu.xlsx", skiprows= 4)

recent_year_for_income= max(list(revenu_med_data.an))

revenu_med_data= revenu_med_data[revenu_med_data.an== recent_year_for_income]
revenu_med_data= revenu_med_data.rename(columns= {"codgeo": "code_com"})

revenu_med_data = revenu_med_data[["code_com", "med_disp"]]

Dataframe_stage_10= Dataframe_stage_9.merge(revenu_med_data, on= "code_com", how= "left") 

Dataframe_stage_10.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,superf_choro,gare_tgv,nb_actes_france_renov,friches,ecoquartiers,part_actifs,part_inactifs,med_disp
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,24.1,0,6.0,2.0,0.0,281,76,20300.0
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,12.2,0,0.0,0.0,0.0,584,128,23570.0
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,6.4,0,8.0,0.0,0.0,453,133,21780.0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,36.4,0,4.0,6.0,0.0,1962,834,21020.0
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,14.1,0,1.0,3.0,0.0,437,146,18770.0


+ **Lecture et jointure des données liée aux parts de residence secondaire part_residences_secondaires**

In [47]:

part_resid_sec_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/PART_RESIDENCES_SECONDAIRES/residence_sec.csv", delimiter= ";")
# 
recent_year_for_residence= max(list(part_resid_sec_data.an))

part_resid_sec_data = part_resid_sec_data[part_resid_sec_data.an== recent_year_for_residence]
part_resid_sec_data = part_resid_sec_data.rename(columns= {"codgeo": "code_com"})

part_resid_sec_data = part_resid_sec_data[["code_com", "part_resid2"]]
part_resid_sec_data.columns = ["code_com", "part_residences_secondaires"]

Dataframe_stage_11= Dataframe_stage_10.merge(part_resid_sec_data, on= "code_com", how= "left") 

Dataframe_stage_11.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,superf_choro,gare_tgv,nb_actes_france_renov,friches,ecoquartiers,part_actifs,part_inactifs,med_disp,part_residences_secondaires
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,24.1,0,6.0,2.0,0.0,281,76,20300.0,12.0
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,12.2,0,0.0,0.0,0.0,584,128,23570.0,2.0
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,6.4,0,8.0,0.0,0.0,453,133,21780.0,3.4
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,36.4,0,4.0,6.0,0.0,1962,834,21020.0,5.5
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,14.1,0,1.0,3.0,0.0,437,146,18770.0,16.4


+ **Lecture et jointure des données liée aux trajet domicile- travails part_trajets_voiture**

In [48]:
deplacement_dom_trav_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/DEPLACEMENT_DOMICILE_TRAVAIL/deplacement_voiture.csv", delimiter= ";")

recent_year_for_deplacement= max(list(deplacement_dom_trav_data.an))

deplacement_dom_trav_data = deplacement_dom_trav_data[deplacement_dom_trav_data.an== recent_year_for_deplacement]
deplacement_dom_trav_data = deplacement_dom_trav_data.rename(columns= {"codgeo": "code_com"})

deplacement_dom_trav_data = deplacement_dom_trav_data[["code_com", "part_domtrav_voit"]]
deplacement_dom_trav_data.columns = ["code_com", "part_trajets_voiture"]

Dataframe_stage_12= Dataframe_stage_11.merge(deplacement_dom_trav_data, on= "code_com", how= "left") 

Dataframe_stage_12.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,superf_choro,gare_tgv,nb_actes_france_renov,friches,ecoquartiers,part_actifs,part_inactifs,med_disp,part_residences_secondaires,part_trajets_voiture
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,24.1,0,6.0,2.0,0.0,281,76,20300.0,12.0,85.7
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,12.2,0,0.0,0.0,0.0,584,128,23570.0,2.0,90.1
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,6.4,0,8.0,0.0,0.0,453,133,21780.0,3.4,91.3
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,36.4,0,4.0,6.0,0.0,1962,834,21020.0,5.5,88.5
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,14.1,0,1.0,3.0,0.0,437,146,18770.0,16.4,79.2


+ **Lecture et jointure des données liée aux endettement de la commune com_variation_encours_dette_ha_pct**

In [49]:
endettement_com_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/ENDETTEMENT/dette_com.csv", delimiter= ";")

recent_year_for_endettement= max(list(endettement_com_data.an))

endettement_com_data = endettement_com_data[endettement_com_data.an== recent_year_for_endettement]
endettement_com_data = endettement_com_data.rename(columns= {"codgeo": "code_com"})

endettement_com_data = endettement_com_data[["code_com", "com_variation_encours_dette_ha_pct"]]
endettement_com_data.columns = ["code_com", "com_variation_encours_dette_ha_pct"]

Dataframe_stage_13= Dataframe_stage_12.merge(endettement_com_data, on= "code_com", how= "left") 

Dataframe_stage_13.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,gare_tgv,nb_actes_france_renov,friches,ecoquartiers,part_actifs,part_inactifs,med_disp,part_residences_secondaires,part_trajets_voiture,com_variation_encours_dette_ha_pct
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,0,6.0,2.0,0.0,281,76,20300.0,12.0,85.7,2.6
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,0,0.0,0.0,0.0,584,128,23570.0,2.0,90.1,-28.7
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,0,8.0,0.0,0.0,453,133,21780.0,3.4,91.3,206.0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,0,4.0,6.0,0.0,1962,834,21020.0,5.5,88.5,7.0
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,0,1.0,3.0,0.0,437,146,18770.0,16.4,79.2,-25.4


+ **Lecture et jointure des données liée aux dépendences eco dependance_eco**

In [50]:
dependance_eco_data = pd.read_excel(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/DEPENDANCE _ECONOMIQUE/dependance_economique.xlsx", skiprows= 4)

recent_year_for_dependance_eco = max(list(dependance_eco_data.an))

dependance_eco_data= dependance_eco_data[dependance_eco_data.an== recent_year_for_dependance_eco]
dependance_eco_data= dependance_eco_data.rename(columns= {"codgeo": "code_com"})

dependance_eco_data= dependance_eco_data[["code_com", "tx_dep_eco"]]
dependance_eco_data.columns = ["code_com", "dependance_eco"]

Dataframe_stage_14= Dataframe_stage_13.merge(dependance_eco_data, on= "code_com", how= "left") 

Dataframe_stage_14.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,nb_actes_france_renov,friches,ecoquartiers,part_actifs,part_inactifs,med_disp,part_residences_secondaires,part_trajets_voiture,com_variation_encours_dette_ha_pct,dependance_eco
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,6.0,2.0,0.0,281,76,20300.0,12.0,85.7,2.6,107.0
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,0.0,0.0,0.0,584,128,23570.0,2.0,90.1,-28.7,96.0
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,8.0,0.0,0.0,453,133,21780.0,3.4,91.3,206.0,113.0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,4.0,6.0,0.0,1962,834,21020.0,5.5,88.5,7.0,126.0
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,1.0,3.0,0.0,437,146,18770.0,16.4,79.2,-25.4,140.0


+ **Lecture et jointure des données liée aux abstentions municipales: abstention_municipales**

In [51]:

abstention_mun_data = pd.read_excel(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/ABSTENTION/abstention_vote.xlsx", skiprows= 4)

recent_year_for_abstention = max(list(abstention_mun_data.an))

abstention_mun_data= abstention_mun_data[abstention_mun_data.an== recent_year_for_abstention]
abstention_mun_data= abstention_mun_data.rename(columns= {"codgeo": "code_com"})

abstention_mun_data= abstention_mun_data[["code_com", "tx_abst_elec_mun"]]
abstention_mun_data.columns = ["code_com", "abstention_municipales"]

Dataframe_stage_15= Dataframe_stage_14.merge(abstention_mun_data, on= "code_com", how= "left") 

Dataframe_stage_15.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,friches,ecoquartiers,part_actifs,part_inactifs,med_disp,part_residences_secondaires,part_trajets_voiture,com_variation_encours_dette_ha_pct,dependance_eco,abstention_municipales
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,2.0,0.0,281,76,20300.0,12.0,85.7,2.6,107.0,17.4
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,0.0,0.0,584,128,23570.0,2.0,90.1,-28.7,96.0,53.8
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,0.0,0.0,453,133,21780.0,3.4,91.3,206.0,113.0,53.7
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,6.0,0.0,1962,834,21020.0,5.5,88.5,7.0,126.0,44.8
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,3.0,0.0,437,146,18770.0,16.4,79.2,-25.4,140.0,22.9


+ **Lecture et jointure des données liée aux taux de création d'établissement : taux_creation_ent**

In [52]:
taux_creation_ent_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/ETABLISSEMENT_CREATION/creation_etablissements.csv", delimiter= ";", skiprows= 2)

lib_for_rename_etab= [col for col in taux_creation_ent_data.columns if col.startswith("Taux de")][0]

taux_creation_ent_data= taux_creation_ent_data.rename(columns= {"Code": "code_com", lib_for_rename_etab : "taux_creation_ent"})

taux_creation_ent_data= taux_creation_ent_data[["code_com", "taux_creation_ent"]]

taux_creation_ent_data= taux_creation_ent_data.replace('N/A - division par 0', pd.NA)

taux_creation_ent_data= taux_creation_ent_data.replace("N/A - résultat non disponible", pd.NA)

taux_creation_ent_data['code_com'] = taux_creation_ent_data['code_com'].astype(str)

taux_creation_ent_data['taux_creation_ent'] = taux_creation_ent_data['taux_creation_ent'].map(lambda x : float(x) if pd.notna(x) else x)

Dataframe_stage_16= Dataframe_stage_15.merge(taux_creation_ent_data, on= "code_com", how= "left") 

Dataframe_stage_16.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,ecoquartiers,part_actifs,part_inactifs,med_disp,part_residences_secondaires,part_trajets_voiture,com_variation_encours_dette_ha_pct,dependance_eco,abstention_municipales,taux_creation_ent
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,0.0,281,76,20300.0,12.0,85.7,2.6,107.0,17.4,21.2
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,0.0,584,128,23570.0,2.0,90.1,-28.7,96.0,53.8,37.3
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,0.0,453,133,21780.0,3.4,91.3,206.0,113.0,53.7,20.0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,0.0,1962,834,21020.0,5.5,88.5,7.0,126.0,44.8,16.0
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,0.0,437,146,18770.0,16.4,79.2,-25.4,140.0,22.9,25.0


+ **Lecture et jointure des données liée aux totale d'établissement: total_entreprises**

In [53]:
tot_entreprises_data = pd.read_excel(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/ETABLISSEMENT_CREATION/tot_creation_etablissements.xlsx", skiprows= 4)

recent_year_for_tot_etab= max(list(tot_entreprises_data.an))

tot_entreprises_data = tot_entreprises_data[tot_entreprises_data.an== recent_year_for_tot_etab]
tot_entreprises_data = tot_entreprises_data.rename(columns= {"codgeo": "code_com"})

tot_entreprises_data = tot_entreprises_data[["code_com", "enntot"]]
tot_entreprises_data.columns = ["code_com", "total_entreprises"]

Dataframe_stage_17= Dataframe_stage_16.merge(tot_entreprises_data, on= "code_com", how= "left") 

Dataframe_stage_17.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,part_actifs,part_inactifs,med_disp,part_residences_secondaires,part_trajets_voiture,com_variation_encours_dette_ha_pct,dependance_eco,abstention_municipales,taux_creation_ent,total_entreprises
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,281,76,20300.0,12.0,85.7,2.6,107.0,17.4,21.2,33.0
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,584,128,23570.0,2.0,90.1,-28.7,96.0,53.8,37.3,51.0
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,453,133,21780.0,3.4,91.3,206.0,113.0,53.7,20.0,40.0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,1962,834,21020.0,5.5,88.5,7.0,126.0,44.8,16.0,281.0
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,437,146,18770.0,16.4,79.2,-25.4,140.0,22.9,25.0,76.0


+ **Lecture et jointure des données liée au part de licencié sportif part_licencies_sportifs**

In [54]:
licencie_sport_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/SPORT/licencie_sportif.csv", delimiter= ";", skiprows= 2)

lib_for_rename_licencie= [col for col in licencie_sport_data.columns if col.startswith("Nombre de")][0]

licencie_sport_data= licencie_sport_data.rename(columns= {"Code": "code_com", lib_for_rename_licencie : "part_licencies_sportifs"})

licencie_sport_data= licencie_sport_data[["code_com", "part_licencies_sportifs"]]

licencie_sport_data['code_com'] = licencie_sport_data['code_com'].astype(str)

Dataframe_stage_18= Dataframe_stage_17.merge(licencie_sport_data, on= "code_com", how= "left") 

Dataframe_stage_18.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,part_inactifs,med_disp,part_residences_secondaires,part_trajets_voiture,com_variation_encours_dette_ha_pct,dependance_eco,abstention_municipales,taux_creation_ent,total_entreprises,part_licencies_sportifs
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,76,20300.0,12.0,85.7,2.6,107.0,17.4,21.2,33.0,144
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,128,23570.0,2.0,90.1,-28.7,96.0,53.8,37.3,51.0,280
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,133,21780.0,3.4,91.3,206.0,113.0,53.7,20.0,40.0,354
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,834,21020.0,5.5,88.5,7.0,126.0,44.8,16.0,281.0,978
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,146,18770.0,16.4,79.2,-25.4,140.0,22.9,25.0,76.0,229


+  **Lecture et jointure des données liée au CSP du maire :CSP_maire**

In [55]:

CSP_maire_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/ELUS CSP/elus_conseillers_municipaux.csv", delimiter= ";")

CSP_maire_data = CSP_maire_data[["Code de la commune","Nom de l'élu", "Prénom de l'élu"]]

CSP_maire_data= CSP_maire_data.rename(columns= {"Code de la commune": "code_com"})

CSP_maire_data["nom_prenom"]= CSP_maire_data["Nom de l'élu"] + " "+ CSP_maire_data["Prénom de l'élu"]

CSP_maire_data= CSP_maire_data[["code_com","nom_prenom"]]


CSP_maire_data_group_1= CSP_maire_data.groupby(["code_com", "nom_prenom"], as_index= False).count()

CSP_maire_data_group_1["CSP_maire"]= 1

CSP_maire_data_group_2= CSP_maire_data_group_1.groupby("code_com", as_index= False).sum("CSP_maire")

CSP_maire_data_group_2['code_com'] = CSP_maire_data_group_2['code_com'].astype(str)

Dataframe_stage_19= Dataframe_stage_18.merge(CSP_maire_data_group_2, on= "code_com", how= "left") 

Dataframe_stage_19.head()

/tmp/ipykernel_399325/512656413.py:1: DtypeWarning: Columns (0,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  CSP_maire_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/ELUS CSP/elus_conseillers_municipaux.csv", delimiter= ";")


,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,med_disp,part_residences_secondaires,part_trajets_voiture,com_variation_encours_dette_ha_pct,dependance_eco,abstention_municipales,taux_creation_ent,total_entreprises,part_licencies_sportifs,CSP_maire
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,20300.0,12.0,85.7,2.6,107.0,17.4,21.2,33.0,144,12
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,23570.0,2.0,90.1,-28.7,96.0,53.8,37.3,51.0,280,15
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,21780.0,3.4,91.3,206.0,113.0,53.7,20.0,40.0,354,12
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,21020.0,5.5,88.5,7.0,126.0,44.8,16.0,281.0,978,27
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,18770.0,16.4,79.2,-25.4,140.0,22.9,25.0,76.0,229,15


+ **Lecture et jointure des données liée au climat climat**

In [56]:
climat_data = pd.read_excel(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/CLIMAT/climat.xlsx",sheet_name= "Perimetre",  skiprows= 2)

recent_year_for_climat = max(list(climat_data["ANNEE_COG"]))

climat_data = climat_data[climat_data["ANNEE_COG"]== recent_year_for_climat]

climat_data = climat_data.rename(columns= {"INSEE_COM": "code_com", "CLASSEMENT" :"climat"})

climat_data= climat_data[["code_com", "climat"]]

Dataframe_stage_20= Dataframe_stage_19.merge(climat_data, on= "code_com", how= "left") 

Dataframe_stage_20['climat']= Dataframe_stage_20['climat'].fillna("Non_riveraine")

Dataframe_stage_20['climat'], nomenclature_climat = pd.factorize(Dataframe_stage_20['climat'])

dict(enumerate(nomenclature_climat))  # nomemclature des modalités


Dataframe_stage_20.head()

/home/emiliano/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,part_residences_secondaires,part_trajets_voiture,com_variation_encours_dette_ha_pct,dependance_eco,abstention_municipales,taux_creation_ent,total_entreprises,part_licencies_sportifs,CSP_maire,climat
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,12.0,85.7,2.6,107.0,17.4,21.2,33.0,144,12,0
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,2.0,90.1,-28.7,96.0,53.8,37.3,51.0,280,15,0
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,3.4,91.3,206.0,113.0,53.7,20.0,40.0,354,12,0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,5.5,88.5,7.0,126.0,44.8,16.0,281.0,978,27,0
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,16.4,79.2,-25.4,140.0,22.9,25.0,76.0,229,15,0


+ **Lecture et jointure des données liée au emission de GES : emission_ges**

In [57]:
emission_ges_data = pd.read_excel(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/EMISSION_GES/emission_ges.xlsx")

emission_ges_data = emission_ges_data.rename(columns= {"INSEE commune": "code_com"})

col_for_sum_ges = [col for col in emission_ges_data.columns if col not in ["code_com", "Commune"]]

emission_ges_data["emissions_ges"]= emission_ges_data[col_for_sum_ges].sum(axis= 1).round(2)

emission_ges_data= emission_ges_data[["code_com", "emissions_ges"]]

Dataframe_stage_21= Dataframe_stage_20.merge(emission_ges_data, on= "code_com", how= "left")

Dataframe_stage_21.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,part_trajets_voiture,com_variation_encours_dette_ha_pct,dependance_eco,abstention_municipales,taux_creation_ent,total_entreprises,part_licencies_sportifs,CSP_maire,climat,emissions_ges
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,85.7,2.6,107.0,17.4,21.2,33.0,144,12,0,15634.38
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,90.1,-28.7,96.0,53.8,37.3,51.0,280,15,0,8233.54
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,91.3,206.0,113.0,53.7,20.0,40.0,354,12,0,6175.22
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,88.5,7.0,126.0,44.8,16.0,281.0,978,27,0,30835.24
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,79.2,-25.4,140.0,22.9,25.0,76.0,229,15,0,7395.71


+ **Lecture et jointure des données liée au benéficiaire du programme de l'ANCT beneficiaire_prog**

In [58]:
#+ Lecture et jointure des données liée au benéficiaire du programme de l'ANCT **beneficiaire_prog**

benef_prog_anct_data = pd.read_excel(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/BENEFICIAIRE_ANCT/beneficiaire_prog_anct.xlsx", skiprows= 4)


benef_prog_anct_data = benef_prog_anct_data.rename(columns= {"codgeo": "code_com"})

benef_prog_anct_data = benef_prog_anct_data[["code_com", "va_zpp"]]

benef_prog_anct_data["beneficiaire_prog"], nomenclature_anct = pd.factorize(benef_prog_anct_data["va_zpp"])

benef_prog_anct_data = benef_prog_anct_data[["code_com", "beneficiaire_prog"]]

Dataframe_stage_22 = Dataframe_stage_21.merge(benef_prog_anct_data, on= "code_com", how= "left") 

dict(enumerate(nomenclature_anct)) #nomenclature des modalitées

Dataframe_stage_22.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,com_variation_encours_dette_ha_pct,dependance_eco,abstention_municipales,taux_creation_ent,total_entreprises,part_licencies_sportifs,CSP_maire,climat,emissions_ges,beneficiaire_prog
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,2.6,107.0,17.4,21.2,33.0,144,12,0,15634.38,0
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,-28.7,96.0,53.8,37.3,51.0,280,15,0,8233.54,0
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,206.0,113.0,53.7,20.0,40.0,354,12,0,6175.22,0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,7.0,126.0,44.8,16.0,281.0,978,27,0,30835.24,0
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,-25.4,140.0,22.9,25.0,76.0,229,15,0,7395.71,0


+ **Lecture et jointure des données liée au part de jeunes sans diplome: part_jeunes_sans_diplome**

In [59]:
sans_diplome_data = pd.read_excel(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/SANS_DIPLOME/sans_diplome.xlsx", skiprows= 4)

recent_year_for_no_diplome= max(list(sans_diplome_data.an))

sans_diplome_data= sans_diplome_data[sans_diplome_data.an== recent_year_for_no_diplome]

sans_diplome_data= sans_diplome_data.rename(columns= {"codgeo": "code_com"})

sans_diplome_data= sans_diplome_data[["code_com", "ss_dip"]]

sans_diplome_data.columns = ["code_com", "part_jeunes_sans_diplome"]

Dataframe_stage_23= Dataframe_stage_22.merge(sans_diplome_data, on= "code_com", how= "left") 

Dataframe_stage_23.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,dependance_eco,abstention_municipales,taux_creation_ent,total_entreprises,part_licencies_sportifs,CSP_maire,climat,emissions_ges,beneficiaire_prog,part_jeunes_sans_diplome
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,107.0,17.4,21.2,33.0,144,12,0,15634.38,0,9.1
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,96.0,53.8,37.3,51.0,280,15,0,8233.54,0,3.9
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,113.0,53.7,20.0,40.0,354,12,0,6175.22,0,0.0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,126.0,44.8,16.0,281.0,978,27,0,30835.24,0,12.4
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,140.0,22.9,25.0,76.0,229,15,0,7395.71,0,2.9


+ **Lecture et jointure des données liée à la création d'entreprise par secteur d'activité , etab_industrie, etab_industrie, etab_CTHR, etab_IC, etab_AFA, etab_AI, etab_ASTSA, etab_APESAS, etab_AAS**


In [60]:
create_ent_data = pd.read_excel(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/ENTREPRISE_PAR_SECTEUR/entreprise_par_secteur.xlsx", skiprows= 3)

create_ent_data = create_ent_data.rename(columns= {"Code": "code_com"})

create_ent_data.columns= [col.replace("Nombre d'établissements par secteurs d'activité 2022_x000d_\n", "").lower() for col in create_ent_data]

create_ent_data= create_ent_data.rename(columns= {"industrie": "etab_industrie",
                                                  "construction": "etab_construction",
                                                  "commerce, transp., héberg. et restauration": "etab_CTHR",
                                                  "information et communication" : "etab_IC",
                                                  "act. financières et assurance" : "etab_AFA",
                                                  "act. immobilières" : "etab_AI",
                                                  "act. scient. & techn., act. de serv. admi." : "etab_ASTSA",
                                                  "adm. publ, enseign, santé, action sociale " : "etab_APESAS",
                                                  "autres act. de services " : "etab_AAS"})

create_ent_data= create_ent_data.drop(columns= "libellé")

Dataframe_stage_24 = Dataframe_stage_23.merge(create_ent_data, on= "code_com", how= "left") 

Dataframe_stage_24.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,part_jeunes_sans_diplome,etab_industrie,etab_construction,etab_CTHR,etab_IC,etab_AFA,etab_AI,etab_ASTSA,etab_APESAS,etab_AAS
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,9.1,2,2,8,0,1,10,4,2,4
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,3.9,6,4,12,0,3,9,5,7,5
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,0.0,0,7,12,0,4,2,9,2,4
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,12.4,16,31,76,3,11,19,37,58,30
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,2.9,8,6,24,0,5,1,8,14,10


+ **Lecture et jointure des données liée aux nombre d'établissement touristique etab_tourisme**

In [61]:
etab_tourisme_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/TOURISME/etablissements_touristiques.csv", delimiter= ";")

etab_tourisme_data= etab_tourisme_data.rename(columns= {"Code postal du professionnel": "code_com", "Nom du professionnel" :"etab_tourisme"})

etab_tourisme_data= etab_tourisme_data[["code_com","etab_tourisme"]].groupby("code_com", as_index=False).count()

etab_tourisme_data['code_com'] = etab_tourisme_data['code_com'].astype(str)

Dataframe_stage_25= Dataframe_stage_24.merge(etab_tourisme_data, on= "code_com", how= "left") 

Dataframe_stage_25["etab_tourisme"]= Dataframe_stage_25["etab_tourisme"].fillna(0)

Dataframe_stage_25.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,etab_industrie,etab_construction,etab_CTHR,etab_IC,etab_AFA,etab_AI,etab_ASTSA,etab_APESAS,etab_AAS,etab_tourisme
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,2,2,8,0,1,10,4,2,4,0.0
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,6,4,12,0,3,9,5,7,5,0.0
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,0,7,12,0,4,2,9,2,4,0.0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,16,31,76,3,11,19,37,58,30,0.0
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,8,6,24,0,5,1,8,14,10,0.0


+ **Lecture et jointure des données liée aux installations de production d'énergie en Bretagne eolienne_PE_kw, eolienne_PT_kw, nbr_eolienne, hydroelectrique_PE_kw, hydroelectrique_PT_kw, nbr_instal_hydroelec, photovoltaique_PE_kw, photovoltaique_PT_kw, nbr_inst_sol_photovoltaique, fossile_PE_kw, fossile_PT_kw, nbr_inst_fossile, incinerateur_PE_kw, incinerateur_PT_kw, nbr_incinerateur, chaufferie_bois_PE_kw, chaufferie_bois_PT_kw, nbr_chaufferie_bois, methaniseur_PE_kw, methaniseur_PT_unite_methaniseur**


In [62]:

prod_energie_data = pd.read_excel(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/PROD_ENERGIE/prod_energie.xlsx")

prod_energie_data = prod_energie_data.rename(columns= {"insee_com": "code_com"})

prod_energie_data = prod_energie_data[["code_com","filiere", "p_elec_kw",	"p_th_kw"]]

prod_energie_data['code_com'] = prod_energie_data['code_com'].astype(str)

# 'Eolienne'
prod_energie_eolienne = prod_energie_data[prod_energie_data["filiere"]== "Eolienne"][["code_com", "p_elec_kw",	"p_th_kw"]]
prod_energie_eolienne.columns= ["code_com","eolienne_PE_kw","eolienne_PT_kw"]
prod_energie_eolienne["nbr_eolienne"]= 1
prod_energie_eolienne = prod_energie_eolienne.groupby("code_com", as_index=False).sum()
Dataframe_stage_26 = Dataframe_stage_25.merge(prod_energie_eolienne, on= "code_com", how= "left") 
Dataframe_stage_26["eolienne_PE_kw"]= Dataframe_stage_26["eolienne_PE_kw"].fillna(0)
Dataframe_stage_26["nbr_eolienne"]= Dataframe_stage_26["nbr_eolienne"].fillna(0)


# 'Installation hydroélectrique'
prod_energie_hydro_elec = prod_energie_data[prod_energie_data["filiere"]== "Installation hydroélectrique"][["code_com", "p_elec_kw",	"p_th_kw"]]
prod_energie_hydro_elec.columns= ["code_com","hydroelectrique_PE_kw","hydroelectrique_PT_kw"]
prod_energie_hydro_elec["nbr_instal_hydroelec"]= 1
prod_energie_hydro_elec = prod_energie_hydro_elec.groupby("code_com", as_index=False).sum()
Dataframe_stage_27 = Dataframe_stage_26.merge(prod_energie_hydro_elec, on= "code_com", how= "left") 
Dataframe_stage_27["hydroelectrique_PE_kw"]= Dataframe_stage_27["hydroelectrique_PE_kw"].fillna(0)
Dataframe_stage_27["nbr_instal_hydroelec"]= Dataframe_stage_27["nbr_instal_hydroelec"].fillna(0)

# 'Installation solaire photovoltaïque'
prod_energie_sol_photovolt = prod_energie_data[prod_energie_data["filiere"]== "Installation solaire photovoltaïque"][["code_com", "p_elec_kw",	"p_th_kw"]]
prod_energie_sol_photovolt.columns= ["code_com","photovoltaique_PE_kw","photovoltaique_PT_kw"]
prod_energie_sol_photovolt["nbr_inst_sol_photovoltaique"]= 1
prod_energie_sol_photovolt = prod_energie_sol_photovolt.groupby("code_com", as_index=False).sum()
Dataframe_stage_28 = Dataframe_stage_27.merge(prod_energie_sol_photovolt, on= "code_com", how= "left") 


# 'Installation fossile'
prod_energie_fossile= prod_energie_data[prod_energie_data["filiere"]== "Installation fossile"][["code_com", "p_elec_kw",	"p_th_kw"]]
prod_energie_fossile.columns= ["code_com","fossile_PE_kw","fossile_PT_kw"]
prod_energie_fossile["nbr_inst_fossile"]= 1
prod_energie_fossile = prod_energie_fossile.groupby("code_com", as_index=False).sum()
Dataframe_stage_29 = Dataframe_stage_28.merge(prod_energie_fossile, on= "code_com", how= "left") 


# 'Incinérateur'
prod_energie_incinerateur= prod_energie_data[prod_energie_data["filiere"]== "Incinérateur"][["code_com", "p_elec_kw",	"p_th_kw"]]
prod_energie_incinerateur.columns= ["code_com","incinerateur_PE_kw", "incinerateur_PT_kw"]
prod_energie_incinerateur["nbr_incinerateur"]= 1
prod_energie_incinerateur = prod_energie_incinerateur.groupby("code_com", as_index=False).sum()
Dataframe_stage_30 = Dataframe_stage_29.merge(prod_energie_incinerateur, on= "code_com", how= "left") 


# 'Chaufferie bois'
prod_energie_chaufferie_bois= prod_energie_data[prod_energie_data["filiere"]== "Chaufferie bois"][["code_com", "p_elec_kw",	"p_th_kw"]]
prod_energie_chaufferie_bois.columns= ["code_com","chaufferie_bois_PE_kw","chaufferie_bois_PT_kw"]
prod_energie_chaufferie_bois["nbr_chaufferie_bois"]= 1
prod_energie_chaufferie_bois = prod_energie_chaufferie_bois.groupby("code_com", as_index=False).sum()
Dataframe_stage_31 = Dataframe_stage_30.merge(prod_energie_chaufferie_bois, on= "code_com", how= "left") 


# 'Unité de méthanisation'
prod_energie_methaniseur= prod_energie_data[prod_energie_data["filiere"]== "Unité de méthanisation"][["code_com", "p_elec_kw",	"p_th_kw"]]
prod_energie_methaniseur.columns= ["code_com","methaniseur_PE_kw","methaniseur_PT_kw"] 
prod_energie_methaniseur["nbr_unite_methaniseur"]= 1
prod_energie_methaniseur = prod_energie_methaniseur.groupby("code_com", as_index=False).sum()
Dataframe_stage_32 = Dataframe_stage_31.merge(prod_energie_methaniseur, on= "code_com", how= "left") 

Dataframe_stage_32[Dataframe_stage_32.columns[47:]]= Dataframe_stage_32[Dataframe_stage_32.columns[47:]].fillna(0)

Dataframe_stage_32.head()

,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,nbr_inst_fossile,incinerateur_PE_kw,incinerateur_PT_kw,nbr_incinerateur,chaufferie_bois_PE_kw,chaufferie_bois_PT_kw,nbr_chaufferie_bois,methaniseur_PE_kw,methaniseur_PT_kw,nbr_unite_methaniseur
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,316.0,360.0,1.0
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,1.0,0.0,0.0,0.0,0.0,1800.0,1.0,0.0,0.0,1.0
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,0.0,0.0,0.0,0.0,0.0,300.0,1.0,0.0,0.0,0.0


+ **Lecture et jointure des données liée à la conformité de la qualité de l'eau  par nombre de prélévement par commune conformite_bacteriologique, conformite_chimique**

In [63]:

# On suppose que la qualité de l'eau à l'échelle bacteriologique ou chimique est conforme si est conforme si la modalité est égale à "S" cad sans objet lorsqu'aucun paramètre chimique n'a été mesuré


qualite_eau_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/CONTROLE_EAU/prelevement_eau.txt", delimiter= ",")

qualite_eau_data = qualite_eau_data.rename(columns= {"inseecommuneprinc": "code_com", "plvconformitereferencebact" : "conformite_bact","plvconformitereferencechim" : "conformite_chim" })

qualite_eau_data = qualite_eau_data[["code_com","conformite_bact",	"conformite_chim"]]

qualite_eau_data['code_com'] = qualite_eau_data['code_com'].astype(str)

qualite_eau_data = qualite_eau_data.replace("S", "C")

qualite_eau_data["total_prelev"] = 1

qualite_eau_data_group_count = qualite_eau_data[["code_com","total_prelev"]].groupby("code_com", as_index=False).sum()

qualite_eau_data = qualite_eau_data.replace("C", 1)
qualite_eau_data = qualite_eau_data.replace("N", 0)

qualite_eau_data_group_sum = qualite_eau_data[["code_com","conformite_bact","conformite_chim"]].groupby("code_com", as_index=False).sum()

qualite_eau_data_final= qualite_eau_data_group_sum.merge(qualite_eau_data_group_count, on= "code_com", how= "left")


qualite_eau_data_final["taux_conformite_bact"] = qualite_eau_data_final["conformite_bact"] / qualite_eau_data_final["total_prelev"]

qualite_eau_data_final["taux_conformite_chim"] = qualite_eau_data_final["conformite_chim"] / qualite_eau_data_final["total_prelev"]

qualite_eau_data_final= qualite_eau_data_final.copy()[["code_com", "taux_conformite_bact", "taux_conformite_chim"]]

Dataframe_stage_33 = Dataframe_stage_32.merge(qualite_eau_data_final, on= "code_com", how= "left") 

Dataframe_stage_33.head()

/tmp/ipykernel_399325/2415482746.py:4: DtypeWarning: Columns (0,1,2,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  qualite_eau_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/CONTROLE_EAU/prelevement_eau.txt", delimiter= ",")
/tmp/ipykernel_399325/2415482746.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  qualite_eau_data = qualite_eau_data.replace("N", 0)


,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,incinerateur_PT_kw,nbr_incinerateur,chaufferie_bois_PE_kw,chaufferie_bois_PT_kw,nbr_chaufferie_bois,methaniseur_PE_kw,methaniseur_PT_kw,nbr_unite_methaniseur,taux_conformite_bact,taux_conformite_chim
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.88,0.76
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,0.0,0.0,0.0,0.0,0.0,316.0,360.0,1.0,1.00,1.00
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,1.00
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,0.0,0.0,0.0,1800.0,1.0,0.0,0.0,1.0,1.00,1.00
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,0.0,0.0,0.0,300.0,1.0,0.0,0.0,0.0,1.00,0.90


+ **Lecture et jointure des données liée aux consommation d'energie MC_agricole_mwh & MC_industrie_mwh & MC_tertiaire_mwh & MC_residentiel_mwh & MC_totale_mwh**


In [64]:
conso_energie_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/CONSO_ENERGIE/conso_energie.csv", delimiter= ";")

conso_energie_data.columns = [col.replace(" ", "_").lower() for col in conso_energie_data.columns]

recent_year_for_conso = max(list(conso_energie_data["année"]))

conso_energie_data = conso_energie_data[conso_energie_data["année"] == recent_year_for_conso]

conso_energie_data= conso_energie_data.rename(columns= {"code_commune": "code_com", "conso_moyenne_(mwh)" : "conso_moyenne_mwh",  "conso_totale_(mwh)" : "conso_totale_mwh"})

conso_energie_data= conso_energie_data.copy()[["code_com", "conso_moyenne_mwh", "conso_totale_mwh", "code_grand_secteur"]]

conso_energie_data['code_com'] = conso_energie_data['code_com'].astype(str)

# INDUSTRIE


conso_data_industrie = conso_energie_data[conso_energie_data["code_grand_secteur"]== "INDUSTRIE"][["code_com", "conso_moyenne_mwh"]]

conso_data_industrie.columns= ["code_com","MC_industrie_mwh"]

conso_data_industrie = conso_data_industrie.groupby("code_com", as_index=False).sum()

Dataframe_stage_34 = Dataframe_stage_33.merge(conso_data_industrie, on= "code_com", how= "left") 

# AGRICULTURE

conso_data_agri = conso_energie_data[conso_energie_data["code_grand_secteur"]== "AGRICULTURE"][["code_com", "conso_moyenne_mwh"]]

conso_data_agri.columns= ["code_com","MC_agricole_mwh"]

conso_data_agri = conso_data_agri.groupby("code_com", as_index=False).sum()

Dataframe_stage_35 = Dataframe_stage_34.merge(conso_data_agri, on= "code_com", how= "left") 

# TERTIAIRE

conso_data_tert = conso_energie_data[conso_energie_data["code_grand_secteur"]== "TERTIAIRE"][["code_com", "conso_moyenne_mwh"]]

conso_data_tert.columns= ["code_com","MC_tertiaire_mwh"]

conso_data_tert = conso_data_tert.groupby("code_com", as_index=False).sum()

Dataframe_stage_36 = Dataframe_stage_35.merge(conso_data_tert, on= "code_com", how= "left") 

# RESIDENTIEL 

conso_data_resid = conso_energie_data[conso_energie_data["code_grand_secteur"]== "RESIDENTIEL"][["code_com", "conso_moyenne_mwh"]]

conso_data_resid.columns= ["code_com","MC_residentiel_mwh"]

conso_data_resid = conso_data_resid.groupby("code_com", as_index=False).sum()

Dataframe_stage_37 = Dataframe_stage_36.merge(conso_data_resid, on= "code_com", how= "left") 

# TOTAL   

conso_data_tot = conso_energie_data[["code_com", "conso_totale_mwh"]]

conso_data_tot.columns= ["code_com","MC_totale_mwh"]

conso_data_tot= conso_data_tot.groupby("code_com", as_index=False).sum()

Dataframe_stage_38 = Dataframe_stage_37.merge(conso_data_tot, on= "code_com", how= "left") 

Dataframe_stage_38.head()


/tmp/ipykernel_399325/1593782943.py:1: DtypeWarning: Columns (3,5,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  conso_energie_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/CONSO_ENERGIE/conso_energie.csv", delimiter= ";")


,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,methaniseur_PE_kw,methaniseur_PT_kw,nbr_unite_methaniseur,taux_conformite_bact,taux_conformite_chim,MC_industrie_mwh,MC_agricole_mwh,MC_tertiaire_mwh,MC_residentiel_mwh,MC_totale_mwh
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,0.0,0.0,0.0,0.88,0.76,5.85700,187.05300,113.320000,5.026000,4126.32500
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,316.0,360.0,1.0,1.00,1.00,292.77800,193.80000,123.678000,5.726000,5701.49500
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,0.0,0.0,0.0,1.00,1.00,26.56100,0.00000,1861.330535,12.070954,4982.62700
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,0.0,0.0,1.0,1.00,1.00,2096.40161,264.87777,4296.855786,14.894277,32948.57484
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,0.0,0.0,0.0,1.00,0.90,389.32300,81.46300,529.544000,4.708000,5061.56400


+ **Jointure des données liée  aux aides subventionnée par l'Etat en Bretagne, par thématique**

In [65]:
fonds_bretagne_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/FONDS_BRETAGNE/fonds_bretagne.csv", delimiter= ",")

fonds_bretagne_data= fonds_bretagne_data.rename(columns= {"beneficiaire_commune_code": "code_com"})

progr_theme_data = pd.read_excel(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/FONDS_BRETAGNE/nomenclature_lfi.xlsx")

progr_theme_data= progr_theme_data[["Programme","Mission"]]

progr_theme_data["Programme"] = progr_theme_data["Programme"].map(lambda x: x.lower().replace("p", ""))

progr_theme_data.columns= ["programme_code", "theme"]

fonds_bretagne_w_theme= fonds_bretagne_data.merge(progr_theme_data, on= "programme_code", how= "left")


fonds_bretagne_w_theme.loc[fonds_bretagne_w_theme["programme_code"] == "ADEME", "theme"] = fonds_bretagne_w_theme.loc[fonds_bretagne_w_theme["programme_code"] == "380", "theme"].iloc[0]

fonds_bretagne_w_theme["theme"]= fonds_bretagne_w_theme["theme"].fillna("Autres fonds")

print(Dataframe_stage_38.code_com.dtype)


def group_by_theme(data, all_code_com, theme_and_col_name_list):

  data_for_return= pd.DataFrame()

  data_for_return["code_com"]= all_code_com

  for th_cl in theme_and_col_name_list:

      data_for_filter= data.loc[data["theme"] == th_cl[0], :]  

      data_for_filter= data_for_filter.drop(columns= ["theme", "programme_code"])

      data_group = data_for_filter.groupby("code_com", as_index=False).sum()
      
      data_group= data_group.rename(columns= {"montant_ae": f"ae_{th_cl[1]}", "montant_cp": f"cp_{th_cl[1]}"})

      data_group["code_com"]= data_group["code_com"].astype(str)

      data_for_return= data_for_return.merge(data_group, on= "code_com", how= "left")
  
  data_for_return= data_for_return.fillna(0)

  return(data_for_return)

theme_and_col_name_list= [
  ["Autres fonds", "AF"],
  ["MAB Administration générale et territoriale de l'État", "AGTE"],
  ["MAC Agriculture, alimentation, forêt et affaires rurales", "AAFAR"],
  ["MAD Aide publique au développement", "APD"],
  ["MCB Culture", "culture"],
  ["MDB Économie", "economie"],
  ["MDC Direction de l'action du Gouvernement", "DAG"],
  ["MEC Enseignement scolaire", "ES"],
  ["MGA Gestion des finances publiques", "GFP"],
  ["MIA Immigration, asile et intégration", "IAI"],
  ["MJA Justice", "justice"],
  ["MMA Médias, livre et industries culturelles", "MLIC"],
  ["MPR Plan de relance", "PR"],
  ["MRA Recherche et enseignement supérieur", "RES"],
  ["MRC Relations avec les collectivités territoriales", "RCT"],
  ["MRD Remboursements et dégrèvements", "RD"],
  ["MSB Sécurités", "securites"],
  ["MSE Solidarité, insertion et égalité des chances", "SIEC"],
  ["MSF Sport, jeunesse et vie associative", "SJVA"],
  ["MTA Écologie, développement et mobilité durables", "EDMD"],
  ["MTB Travail, emploi et administration des ministères sociaux", "TEAMS"],
  ["MTR Transformation et fonction publiques", "TFP"],
  ["MVA Cohésion des territoires", "CT"],
  ["MYB Gestion du patrimoine immobilier de l'État", "GPIE"],
  ["MYE Contrôle de la circulation et du stationnement routiers", "CCSR"]
]


fonds_adm_gen_ter_etat = group_by_theme(data= fonds_bretagne_w_theme, all_code_com= Dataframe_stage_38.code_com, theme_and_col_name_list= theme_and_col_name_list)

# Taux de projet subventionner par commune 
montant_cp_col= [val for val in fonds_adm_gen_ter_etat.columns if val.startswith("cp_")]

fonds_adm_gen_ter_etat["taux_subvention"] = (fonds_adm_gen_ter_etat[montant_cp_col] > 0).sum(axis=1) / len(montant_cp_col)

Dataframe_stage_39= Dataframe_stage_38.merge(fonds_adm_gen_ter_etat, on= "code_com", how= "left")

Dataframe_stage_39


object


,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,ae_TEAMS,cp_TFP,ae_TFP,cp_CT,ae_CT,cp_GPIE,ae_GPIE,cp_CCSR,ae_CCSR,taux_subvention
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,0.00,0.00,0.00,0.00,0.000000e+00,0.0,0.0,35368.0,35368.0,0.16
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,0.00,0.00,0.00,56081.55,5.608155e+04,0.0,0.0,0.0,0.0,0.08
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,0.00,0.00,0.00,0.00,0.000000e+00,0.0,0.0,0.0,0.0,0.20
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,0.00,0.00,0.00,0.00,0.000000e+00,0.0,0.0,0.0,0.0,0.40
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,0.00,0.00,0.00,3325000.00,3.325000e+06,0.0,0.0,0.0,0.0,0.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,56260,53.0,56,563,Vannes,Vannes,Morbihan,Bretagne,1,54420.0,...,35578519.47,227108.61,227108.68,89472394.38,2.759091e+08,149677.0,256937.9,3124214.0,3124214.0,0.88
1198,56261,53.0,56,563,La Vraie-Croix,Vannes,Morbihan,Bretagne,5,1474.0,...,0.00,0.00,0.00,0.00,0.000000e+00,0.0,0.0,39300.0,39300.0,0.12
1199,56262,53.0,56,563,Le Bono,Vannes,Morbihan,Bretagne,5,2581.0,...,0.00,0.00,0.00,0.00,0.000000e+00,0.0,0.0,15108.0,15108.0,0.16
1200,56263,53.0,56,561,Sainte-Anne-d'Auray,Lorient,Morbihan,Bretagne,5,2827.0,...,0.00,0.00,0.00,0.00,0.000000e+00,0.0,0.0,39300.0,39300.0,0.32


+ **Jointure des données liée  aux types des établissement autres ayant rerçu des aides subventionnée par l'Etat en Bretagne**

In [66]:
autres_fonds_group_etab_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/FONDS_BRETAGNE/all_fonds_group_on_code_com_type_etab.csv", delimiter= ",")


autres_fonds_group_etab_data = autres_fonds_group_etab_data.rename(
    columns= {"beneficiaire_commune_code" : "code_com",
              "beneficiaire_categorieJuridique_type": "type_etablissement_benef",
              "total_etablissment_type" : "total_etablissement_finance"})

autres_fonds_group_etab_data = pd.read_csv(f"/home/emiliano/Documents/MASTER_MAS/ALTERNANCE/FONDS VERT/DATA/FONDS_BRETAGNE/all_fonds_group_on_code_com_type_etab.csv", delimiter= ",")


autres_fonds_group_etab_data = autres_fonds_group_etab_data.rename(columns= {"beneficiaire_commune_code" : "code_com",
"beneficiaire_categorieJuridique_type": "type_etablissement_benef",
"total_etablissment_type" : "total_etablissement_finance"})

autres_fonds_group_etab_data['code_com'] = autres_fonds_group_etab_data['code_com'].astype(str)


#'Collectivité'
autres_benef_collectivite_data = autres_fonds_group_etab_data[autres_fonds_group_etab_data.type_etablissement_benef=="Collectivité"][["code_com" ,  "total_etablissement_finance"]]
autres_benef_collectivite_data.columns= ["code_com" ,"autres_benef_collectivite"]

Dataframe_stage_40= Dataframe_stage_39.merge(autres_benef_collectivite_data, on= "code_com", how= "left") 
Dataframe_stage_40["autres_benef_collectivite"]= Dataframe_stage_40["autres_benef_collectivite"].fillna(0)


#'Entreprise'
autres_benef_entreprise_data = autres_fonds_group_etab_data[autres_fonds_group_etab_data.type_etablissement_benef=="Entreprise"][["code_com" ,  "total_etablissement_finance"]]
autres_benef_entreprise_data.columns= ["code_com" ,"autres_benef_entreprise"]

Dataframe_stage_41= Dataframe_stage_40.merge(autres_benef_entreprise_data, on= "code_com", how= "left") 
Dataframe_stage_41["autres_benef_entreprise"]= Dataframe_stage_41["autres_benef_entreprise"].fillna(0)

#'Etat'
autres_benef_etat_data = autres_fonds_group_etab_data[autres_fonds_group_etab_data.type_etablissement_benef=="Etat"][["code_com" ,  "total_etablissement_finance"]]
autres_benef_etat_data.columns= ["code_com" ,"autres_benef_etat"]

Dataframe_stage_42= Dataframe_stage_41.merge(autres_benef_etat_data, on= "code_com", how= "left") 
Dataframe_stage_42["autres_benef_etat"]= Dataframe_stage_42["autres_benef_etat"].fillna(0)
#'Association'
autres_benef_association_data = autres_fonds_group_etab_data[autres_fonds_group_etab_data.type_etablissement_benef=="Association"][["code_com" ,  "total_etablissement_finance"]]
autres_benef_association_data.columns= ["code_com" ,"autres_benef_association"]

Dataframe_stage_final= Dataframe_stage_42.merge(autres_benef_association_data, on= "code_com", how= "left") 
Dataframe_stage_final["autres_benef_association"]= Dataframe_stage_final["autres_benef_association"].fillna(0)


autres_fonds_group_etab_data['code_com'] = autres_fonds_group_etab_data['code_com'].astype(str)


#'Collectivité'
autres_benef_collectivite_data = autres_fonds_group_etab_data[autres_fonds_group_etab_data.type_etablissement_benef=="Collectivité"][["code_com" ,  "total_etablissement_finance"]]
autres_benef_collectivite_data.columns= ["code_com" ,"autres_benef_collectivite"]

Dataframe_stage_40= Dataframe_stage_39.merge(autres_benef_collectivite_data, on= "code_com", how= "left") 
Dataframe_stage_40["autres_benef_collectivite"]= Dataframe_stage_40["autres_benef_collectivite"].fillna(0)


#'Entreprise'
autres_benef_entreprise_data = autres_fonds_group_etab_data[autres_fonds_group_etab_data.type_etablissement_benef=="Entreprise"][["code_com" ,  "total_etablissement_finance"]]
autres_benef_entreprise_data.columns= ["code_com" ,"autres_benef_entreprise"]

Dataframe_stage_41= Dataframe_stage_40.merge(autres_benef_entreprise_data, on= "code_com", how= "left") 
Dataframe_stage_41["autres_benef_entreprise"]= Dataframe_stage_41["autres_benef_entreprise"].fillna(0)

#'Etat'
autres_benef_etat_data = autres_fonds_group_etab_data[autres_fonds_group_etab_data.type_etablissement_benef=="Etat"][["code_com" ,  "total_etablissement_finance"]]
autres_benef_etat_data.columns= ["code_com" ,"autres_benef_etat"]

Dataframe_stage_42= Dataframe_stage_41.merge(autres_benef_etat_data, on= "code_com", how= "left") 
Dataframe_stage_42["autres_benef_etat"]= Dataframe_stage_42["autres_benef_etat"].fillna(0)
#'Association'
autres_benef_association_data = autres_fonds_group_etab_data[autres_fonds_group_etab_data.type_etablissement_benef=="Association"][["code_com" ,  "total_etablissement_finance"]]
autres_benef_association_data.columns= ["code_com" ,"autres_benef_association"]

Dataframe_stage_final= Dataframe_stage_42.merge(autres_benef_association_data, on= "code_com", how= "left") 
Dataframe_stage_final["autres_benef_association"]= Dataframe_stage_final["autres_benef_association"].fillna(0)


+ **Traitement final et Présentation de la table final**

In [67]:
# Reformatage des données manquantes

Dataframe_stage_final= Dataframe_stage_final.fillna(pd.NA)

Dataframe_stage_final.replace("N/A - résultat non disponible", pd.NA, inplace=True)


# Suppression des colonnes qui n'apportent pas d'info

# Liste des colonnes à exclure
excluded_columns = ["code_com", "code_reg", "code_dept", "code_arr", 
                    "lib_com", "lib_arr", "lib_dept", "lib_reg"]

# Colonnes à tester (celles qui ne sont pas exclues)
cols_to_check = [col for col in Dataframe_stage_final.columns if col not in excluded_columns]

# On garde :
# - les colonnes exclues
# - les colonnes dont la somme des valeurs numériques est > 0
cols_to_keep = excluded_columns + [
    col for col in cols_to_check 
    if pd.to_numeric(Dataframe_stage_final[col], errors='coerce').sum() > 0
]

# Filtrage du DataFrame
Dataframe_stage_final = Dataframe_stage_final[cols_to_keep]

Dataframe_stage_final.shape


Dataframe_stage_final.to_csv(f"fonds_vert.csv", index=False)

Dataframe_stage_final.head()

/tmp/ipykernel_399325/4272992001.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Dataframe_stage_final= Dataframe_stage_final.fillna(pd.NA)


,code_com,code_reg,code_dept,code_arr,lib_com,lib_arr,lib_dept,lib_reg,gridens7,p_pop,...,ae_CT,cp_GPIE,ae_GPIE,cp_CCSR,ae_CCSR,taux_subvention,autres_benef_collectivite,autres_benef_entreprise,autres_benef_etat,autres_benef_association
0,22001,53.0,22,224,Allineuc,Saint-Brieuc,Côtes-d'Armor,Bretagne,7,596.0,...,0.00,0.0,0.0,35368.0,35368.0,0.16,2.0,33.0,0.0,0.0
1,22002,53.0,22,224,Andel,Saint-Brieuc,Côtes-d'Armor,Bretagne,5,1156.0,...,56081.55,0.0,0.0,0.0,0.0,0.08,1.0,16.0,0.0,0.0
2,22003,53.0,22,221,Aucaleuc,Dinan,Côtes-d'Armor,Bretagne,6,929.0,...,0.00,0.0,0.0,0.0,0.0,0.20,1.0,1.0,0.0,3.0
3,22004,53.0,22,222,Bégard,Guingamp,Côtes-d'Armor,Bretagne,5,4839.0,...,0.00,0.0,0.0,0.0,0.0,0.40,14.0,37.0,0.0,8.0
4,22005,53.0,22,222,Belle-Isle-en-Terre,Guingamp,Côtes-d'Armor,Bretagne,5,1030.0,...,3325000.00,0.0,0.0,0.0,0.0,0.36,10.0,6.0,0.0,6.0


Nous obtenons après constitution de la base de données et prétraitement un ensemble de 1202 commune et 120 variables parmi lesquelle nous relevons une variable à prédire ou variable cible "benef_trans_eco", et 29 variables qualitatives discrete, 83 variable quantitative(discrete & continue ) et 8 variables relative au COG 


In [68]:
glossaire_cigle= {
                  "Activité financieres_assurance" : "AFA",
                  "Activité immobilieres": "AI",
                  "Activité scientifique, technologique et de services administratives": "ASTSA",
                  "Administration publique d'enseignement, sante et action sociale": "APESAS",
                  "Administration générale et territoriale de l'État": "AGTE",
                  "Agriculture, alimentation, forêt et affaires rurales": "AAFAR",
                  "Aide publique au développement": "APD",
                  "Autres fonds": "AF",
                  "Commerce , transport , hebergement et restauration" : "CTHR",
                  "Direction de l'action du Gouvernement": "DAG",
                  "Enseignement scolaire": "ES",
                  "Gestion des finances publiques": "GFP",
                  "Immigration, asile et intégration": "IAI",
                  "Information et communication": "IC",                  
                  "Médias, livre et industries culturelles": "MLIC",
                  "Moyenne des consommations": "MC",
                  "Plan de relance": "PR",
                  "Production thermique": "PT",
                  "Production éléctrique": "PE",
                  "Recherche et enseignement supérieur": "RES",
                  "Relations avec les collectivités territoriales": "RCT",
                  "Remboursements et dégrèvements": "RD",
                  "Solidarité, insertion et égalité des chances": "SIEC",
                  "Sport, jeunesse et vie associative": "SJVA",
                  "Écologie, développement et mobilité durables": "EDME",
                  "Travail, emploi et administration des ministères sociaux": "TEAMS",
                  "Transformation et fonction publiques": "TFP",
                  "Cohésion des territoires": "CT",
                  "Gestion du patrimoine immobilier de l'État": "GPIE",
                  "Contrôle de la circulation et du stationnement routiers": "CCSR"}






import pandas as pd
df_cigle = pd.DataFrame(list(glossaire_cigle.items()), columns=["Intitulé", "Cigle"])

df_cigle

,Intitulé,Cigle
0,Activité financieres_assurance,AFA
1,Activité immobilieres,AI
2,"Activité scientifique, technologique et de ser...",ASTSA
3,"Administration publique d'enseignement, sante ...",APESAS
4,Administration générale et territoriale de l'État,AGTE
5,"Agriculture, alimentation, forêt et affaires r...",AAFAR
6,Aide publique au développement,APD
7,Autres fonds,AF
8,"Commerce , transport , hebergement et restaura...",CTHR
9,Direction de l'action du Gouvernement,DAG
